# FSL course - BET

This interactive demonstration is based on the official FSL course **"Introductory FSL Practicals/ Introduction - BET"**.

Brain extraction is a fundamental preprocessing step in neuroimaging analysis, particularly critical for structural image segmentation where precision matters most. While BET is straightforward to use, achieving optimal results often requires understanding how to fine-tune parameters for challenging datasets.
This interactive version covers the core BET fundamentals, including parameter adjustment techniques for difficult images, and troubleshooting approaches for problematic cases. The hands-on format allows you to experiment with different settings and immediately see their effects on brain extraction quality.

**Author:** Monika Doerig

**Citation/ Resources:** 
- Jenkinson, M., Beckmann, C. F., Behrens, T. E. J., Woolrich, M. W., & Smith, S. M. (2012). FSL. NeuroImage, 62(2), 782–790. [https://doi.org/10.1016/j.neuroimage.2011.09.015](https://doi.org/10.1016/j.neuroimage.2011.09.015)

- [FSL course online materials](https://open.win.ox.ac.uk/pages/fslcourse/website/online_materials.html)
- [FSL course practicals BET](https://open.win.ox.ac.uk/pages/fslcourse/practicals/intro2/index.html)
- [FSL Wiki BET](https://fsl.fmrib.ox.ac.uk/fsl/docs/#/structural/bet?id=bet-brain-extraction-tool)

### Output CPU information

In [1]:
!cat /proc/cpuinfo | grep 'vendor' | uniq
!cat /proc/cpuinfo | grep 'model name' | uniq

vendor_id	: AuthenticAMD
model name	: AMD EPYC-Milan Processor


In [2]:
import module
await module.load('fsl/6.0.7.16') # specific version
await module.list()

['fsl/6.0.7.16']

## Download course material

In [3]:
!wget -c -nc https://fsl.fmrib.ox.ac.uk/fslcourse/downloads/preCourse.tar.gz
!tar -xzvf preCourse.tar.gz --skip-old-files

--2025-12-16 19:00:17--  https://fsl.fmrib.ox.ac.uk/fslcourse/downloads/preCourse.tar.gz
Resolving fsl.fmrib.ox.ac.uk (fsl.fmrib.ox.ac.uk)... 129.67.248.66
Connecting to fsl.fmrib.ox.ac.uk (fsl.fmrib.ox.ac.uk)|129.67.248.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239640893 (229M) [application/x-gzip]
Saving to: ‘preCourse.tar.gz’

preCourse.tar.gz    100%[===================>] 228.54M  31.0MB/s    in 8.1s    

2025-12-16 19:00:26 (28.3 MB/s) - ‘preCourse.tar.gz’ saved [239640893/239640893]

fsl_course_data/intro/
fsl_course_data/intro/highres.nii.gz
fsl_course_data/intro/bighead.nii.gz
fsl_course_data/intro/filtered_func_data.nii.gz
fsl_course_data/intro/example_func.nii.gz
fsl_course_data/intro/structural.nii.gz
fsl_course_data/intro/egfmri.nii.gz
fsl_course_data/intro/example_func2highres.mat
fsl_course_data/intro/diffdata.nii.gz
fsl_course_data/intro/image0.nii.gz
fsl_course_data/intro/egepi.nii.gz
fsl_course_data/intro/standard.nii.gz
fsl_course_da

## BET basics
BET performs brain extraction by removing non-brain tissue from structural MRI images:

`bet <input> <output> [options]`

- Input: Structural image (e.g., structural.nii.gz)
- Output: Brain-extracted image
- Options: Additional outputs like binary mask or skull surface (optional)

For detailed instructions, see the complete FSL tutorial and the help page.

In [4]:
!bet


Usage:    bet <input> <output> [options]

Main bet2 options:
  -o          generate brain surface outline overlaid onto original image
  -m          generate binary brain mask
  -s          generate approximate skull image
  -n          don't generate segmented brain image output
  -f <f>      fractional intensity threshold (0->1); default=0.5; smaller values give larger brain outline estimates
  -g <g>      vertical gradient in fractional intensity threshold (-1->1); default=0; positive values give larger brain outline at bottom, smaller at top
  -r <r>      head radius (mm not voxels); initial surface sphere is set to half of this
  -c <x y z>  centre-of-gravity (voxels not mm) of initial mesh surface.
  -t          apply thresholding to segmented brain image and mask
  -e          generates brain surface as mesh in .vtk format

Variations on default bet2 functionality (mutually exclusive options):
  (default)   just run bet2
  -R          robust brain centre estimation (iterates BE

In [10]:
! mkdir -p output
! bet ./fsl_course_data/intro/structural.nii.gz ./output/structural_brain -m -s

In [11]:
! ls output -lrt

total 3620
-rw-r--r-- 1 jovyan users 3516911 Dec 16 19:05 structural_brain.nii.gz
-rw-r--r-- 1 jovyan users   73650 Dec 16 19:05 structural_brain_mask.nii.gz
-rw-r--r-- 1 jovyan users  114340 Dec 16 19:05 structural_brain_skull.nii.gz


### Visualization with ipyniivue

In [8]:
from ipyniivue import NiiVue
nv = NiiVue()
nv.load_volumes( [{"path": "./fsl_course_data/intro/structural.nii.gz", "colormap": "gray"}])
nv


NiiVue(height=300)

In [7]:
from ipyniivue import NiiVue
nv = NiiVue()
nv.load_volumes( [{"path": "./fsl_course_data/intro/structural.nii.gz", "colormap": "gray"},
                {"path": "./output/structural_brain.nii.gz", "colormap": "red" },
                 {"path": "./output/structural_brain_skull.nii.gz", "colormap": "blue" }])
nv

FileNotFoundError: [Errno 2] No such file or directory: 'output/structural_brain.nii.gz'

In [ ]:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/NeuroDesk/example-notebooks/refs/heads/main/books/images/fsl_course_bet05.png')

### Varying the fractional intensity threshold parameter (-f)
The fractional intensity threshold is BET's key parameter for distinguishing brain tissue from non-brain tissue. This interactive demonstration shows how different `-f` values affect brain extraction results:

- Lower values (e.g., -f 0.2): More inclusive extraction - captures more tissue but may include non-brain areas
- Higher values (e.g., -f 0.8): More conservative extraction - tighter brain boundary but may exclude brain tissue
- Default (-f 0.5): Balanced approach suitable for most cases

The folowing visualization shows three different threshold results overlaid on the original image, demonstrating how this single parameter dramatically changes the extraction outcome.

In [ ]:
! bet ./fsl_course_data/intro/structural.nii.gz ./output/structural_brain_f02 -m -s -f 0.2
! bet ./fsl_course_data/intro/structural.nii.gz ./output/structural_brain_f08 -m -s -f 0.8

In [ ]:
nv = NiiVue()
nv.load_volumes( [{"path": "./output/structural_brain_f02.nii.gz", "colormap": "gray"},
                 {"path": "./output/structural_brain.nii.gz", "colormap": "green"}, 
                  {"path": "./output/structural_brain_f08.nii.gz", "colormap": "red"}])
nv

In [ ]:
Image(url='https://raw.githubusercontent.com/NeuroDesk/example-notebooks/refs/heads/main/books/images/fsl_course_bet_f.png')

## Troubleshooting Brain Extraction

### Cerebellum Underestimation: Using the gradient threshold option (-g)

- **Problem:** Lower brain regions (cerebellum) get cut off


In [ ]:
! bet ./fsl_course_data/intro/sub3m0.nii.gz ./output/sub3m0_brain

In [ ]:
nv = NiiVue()
nv.load_volumes( [{"path": "./output/sub3m0_brain.nii.gz"}])
nv

In [ ]:
Image(url='https://raw.githubusercontent.com/NeuroDesk/example-notebooks/refs/heads/main/books/images/fsl_course_bet_sub3m0.png')

- **Solution:** Use gradient threshold (-g) to vary the intensity threshold linearly by slice (getting smaller at the bottom and bigger at the top, or vice versa)
- **Example:** Try -f 0.3 -g 0.2 for balanced results

In [ ]:
! bet ./fsl_course_data/intro/sub3m0.nii.gz ./output/sub3m0_f03g02 -f 0.3 -g 0.2

In [ ]:
nv = NiiVue()
nv.load_volumes([{"path": "./output/sub3m0_f03g02.nii.gz", "colormap": "gray"},
                 {"path": "./output/sub3m0_brain.nii.gz", "colormap": "green"}])
nv

In [ ]:
Image(url='https://raw.githubusercontent.com/NeuroDesk/example-notebooks/refs/heads/main/books/images/fsl_course_bet_sub3m0_g.png')

### Dealing with large FOV images

- **Problem:** Large amount of neck/extra tissue confuses brain detection (initial brain surface is initialised too low):

In [ ]:
! bet ./fsl_course_data/intro/bighead.nii.gz ./output/bighead_brain

In [ ]:
nv = NiiVue()
nv.load_volumes( [{"path": "./fsl_course_data/intro/bighead.nii.gz", "colormap": "gray"},
                  {"path": "./output/bighead_brain.nii.gz", "colormap": "red"}])

nv

In [ ]:
Image(url='https://raw.githubusercontent.com/NeuroDesk/example-notebooks/refs/heads/main/books/images/fsl_course_bet_bighead.png')

- **Solutions:**

- Crop the image first to remove the neck: `robustfov -i input -r output_crop` (recommended)
- Leave large FOV, but set brain centre-of-gravity: `-c <x y z> option`  (manual coordinate specification)
- Robust brain centre estimation: `-R   option` (iterates BET several times)

Different images may need different approaches - having multiple strategies available is essential for handling problematic cases.

**Recommended Practice**: Combine cropping with `-R` option for most robust results across different image types:

In [ ]:
# crop the image
!robustfov -i ./fsl_course_data/intro/bighead.nii.gz -r ./output/bighead_crop

In [ ]:
nv = NiiVue()
nv.load_volumes( [{"path": "./fsl_course_data/intro/bighead.nii.gz", "colormap": "gray", "opacity": 1.0},
                  {"path": "./output/bighead_crop.nii.gz", "colormap": "blue", "opacity": 1.0}])

nv

In [ ]:
Image(url='https://raw.githubusercontent.com/NeuroDesk/example-notebooks/refs/heads/main/books/images/fsl_course_bet_bighead_crop.png')

In [ ]:
! bet ./output/bighead_crop.nii.gz ./output/bighead_crop_brain -R

In [ ]:
nv = NiiVue()
nv.load_volumes( [{"path": "./output/bighead_brain.nii.gz", "colormap": "gray"},
                  {"path": "./output/bighead_crop_brain.nii.gz", "colormap": "red"}])

nv

In [ ]:
Image(url='https://raw.githubusercontent.com/NeuroDesk/example-notebooks/refs/heads/main/books/images/fsl_course_bet_bighead_crop_brain.png')